In [1]:
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from mlProject.logging.customLogger import logger
from sklearn.model_selection import train_test_split

In [2]:
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow/notebooks


In [3]:
os.chdir('../')

In [4]:
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow


In [5]:
# 1. Update config.yaml
# 4. Update the entity
@dataclass(frozen=True)
class DataTransformationConfig:
  root_dir: Path
  data_path: Path

In [6]:
# 5. Update configuration manager in src config
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH

(PosixPath('config/config.yaml'),
 PosixPath('params.yaml'),
 PosixPath('schema.yaml'))

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
# 6. Update the components
class DataTransformation:
  def __init__(self, config: DataTransformationConfig):
    self.config = config

  # Note: We can add different data transformations such as Scaler, Onehot encoding etc.
  # All kinds of EDA in ML cycle can also be passed here before passing data to the model.
  # Only train test split is added here because data is already clean and ready for use.
    
  def train_test_splitting(self):
    data = pd.read_csv(self.config.data_path)
    # train test split (0.80, 0.20)
    train, test = train_test_split(data)

    train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
    test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

    logger.info("Splitted data into train and test sets.")
    logger.info(train.shape)
    logger.info(test.shape)

    print(train.shape)
    print(test.shape)

In [9]:
# 7. Update pipleline
try:
  config = ConfigurationManager()
  data_transformation_config = config.get_data_transformation_config()
  data_transformation = DataTransformation(config=data_transformation_config)
  data_transformation.train_test_splitting()
  
except Exception as e:
  raise e

Mar 20 12:51:39 common:65 INFO YAML file: config/config.yaml loaded successfully!
Mar 20 12:51:39 common:65 INFO YAML file: params.yaml loaded successfully!
Mar 20 12:51:39 common:65 INFO YAML file: schema.yaml loaded successfully!
Mar 20 12:51:39 common:29 INFO Directory created at: artifacts
Mar 20 12:51:39 common:29 INFO Directory created at: artifacts/data_transformation
Mar 20 12:51:39 4177759729:18 INFO Splitted data into train and test sets.
Mar 20 12:51:39 4177759729:19 INFO (1199, 12)
Mar 20 12:51:39 4177759729:20 INFO (400, 12)
(1199, 12)
(400, 12)
